# Finite Element Modeling with ANSYS

### Overview

<blockquote>
︎❖ *Objective*:  Understand course expectations and policies.  Understand the underlying computational process of the finite element method.
</blockquote>

-   Review the syllabus.

<blockquote>
"An FEA program allows an engineer to make mistakes at a rapid rate of speed. (R. Miller)"
</blockquote>

This is important, because a popular tenet of current design thought suggests that you should fail often and fail cheaply—in other words, bring your model in contact with reality as often as you can sustain in order to exploit the corrections that come from 

To a certain extent, I want you to treat everything I suggest in this class as a set of guidelines which can be explored and challenged.  Think of the computational environment as itself empirical—it is often easy for you to test small hypotheses and craft working problems which can be compared to each other and to reality.  Many questions in the course may thus be answered by the rejoinder—*do it!*

# A Brief Review of the Finite Element Method

### Solution Process

The basic algorithm for a finite element solution is as follows:

1.  Decompose the elements into structures and write a governing equation relationship (such as force-displacement) for each element.
1.  Assemble all of the elements to get the governing equation statement for the entire problem, $\underline{\underline{K}} \underline{c} = \underline{F}$.
1.  Apply the boundary conditions, $\underline{\underline{K}}^{*} \underline{c} = \underline{F}^{*}$.
1.  Solve the resulting linear system to get the solution $\underline{c}$.
1.  Post-process this result to extract stress, strain, displacement, and other variables of interest.

Mathematically, we proceed a bit differently, but it is easier to see where the foregoing steps arise if you have an understanding of the four basic forms of the problem:  strong $\mathbb{S}$, weak $\mathbb{W}$, Galerkin $\mathbb{G}$, and matrix $\mathbb{M}$.

#### $\mathbb{S}$ Strong Form

The strong form is the canonical mathematical statement of the boundary-value problem which you are accustomed to from your previous engineering studies.  The governing equations are stated for the domain, along with boundary conditions.

<blockquote>
Given $f$, $g$, $h$, find $u(x)$ such that
$$
\begin{align}
\frac{\text{d}^{2} u(x)}{\text{d} x^{2}} + f & = 0 & 0<x<1 \\
u(1) & = g \\
\left. \frac{\text{d} u}{\text{d} x} \right|_{x=0} & = -h \\
\end{align}
$$
</blockquote>

#### $\mathbb{W}$ Weak Form

The weak form is an equivalent variational statement of the problem.  A solution of $\mathbb{S}$ satisfies $\mathbb{W}$, and vice versa.

Integrate by parts to construct the weak form.  The weighting functions $w(x)$ are arbitrary as long as they satisfy the homogeneous form of the essential boundary conditions:  $w(x=1) = 0$.  Then the weak statement of the problem is:

<blockquote>
For any $w(x)$ such that $w(x=1)=0$, find $u(x)$ such that the following equation is satisfied:
$$
\begin{align}
\int_{0}^{1} \text{d}x\, \frac{\text{d}w}{\text{d}x} \frac{\text{d}u}{\text{d}x}
& = \left. w(x) \frac{\text{d}^{2} u(x)}{\text{d} x^{2}} \right|_{x=0}^{1} - \int_{0}^{1} \text{d}x\, \frac{\text{d}^{2} u(x)}{\text{d} x^{2}} w(x) \\
& = - w(x) f(x) \bigg|_{x=0}^{1} + \int_{0}^{1} \text{d}x\, f(x) w(x) \hspace{1cm}\text{by}\; \frac{\text{d}^{2} u(x)}{\text{d} x^{2}} + f = 0 \\
\int_{0}^{1} \text{d}x\, \frac{\text{d}w}{\text{d}x} \frac{\text{d}u}{\text{d}x}
& = \int_{0}^{1} \text{d}x\, w(x) f(x) + h w(0)
\end{align}
$$
</blockquote>

#### $\mathbb{G}$ Galerkin Form

Although the matrix form $\mathbb{M}$ is the actual statement of the problem which is solved by the machine, it is the Galerkin form $\mathbb{G}$ which dominates the numerics.  At this point, we assume a discretization (a mesh) and make approximations as to how we will represent the components of the solution.

Discretize the domain and approximate $u$ and $w$ by $\tilde{u}$ and $\tilde{w}$.

<blockquote>
For any $\tilde{w}(x)$ such that $\tilde{w}(x=1)=0$, find $\tilde{u}(x)$ such that the following equation is satisfied:
$$
\begin{align}
\int_{0}^{1} \text{d}x\, \frac{\text{d}w}{\text{d}x} \frac{\text{d}u}{\text{d}x}
& = \int_{0}^{1} \text{d}x\, w(x) f(x) + h w(0)
\Rightarrow \\
\int_{0}^{1} \text{d}x\, \frac{\text{d}\tilde{w}}{\text{d}x} \frac{\text{d}\tilde{u}}{\text{d}x}
& = \int_{0}^{1} \text{d}x\, \tilde{w}(x) f(x) + h \tilde{w}(0)
\end{align}
$$
</blockquote>

#### $\mathbb{M}$ Matrix Form

Although the Galerkin form is approximate, we have not yet specified the discretized approximating functions $\tilde{u}$ and $\tilde{w}$.  We will treat these as composed of shape functions $N_i$, which are equal to zero outside of the interval $x_{i-1} \leq x \leq x_{i+1}$.  The form of the shape functions can be varied, but we will here use a linear form:
$$
N_i(x) = \left\{
\begin{matrix}
\frac{x-x_{i-1}}{h_{i}}
\hspace{1cm} & x_{i-1} \leq x \leq x_{i} \\
\frac{x_{i+1}-x}{h_{i}}
\hspace{1cm} & x_{i} \leq x \leq x_{i+1} \\
0 & \text{elsewhere}
\end{matrix}
\right.
\text{.}
$$

![](./img/fea/shape-fns-linear.png)

The given approximating functions use these as a basis:
$$
\begin{align}
\tilde{u}(x)
& = \sum_{i=1}^{n+1} c_{i} N_{i}(x) \\
& = \sum_{i=1}^{n}   c_{i} N_{i}(x) + g N_{n+1}(x) \\ \\
\tilde{w}(x)
& = \sum_{i=1}^{n+1} d_{i} N_{i}(x) \\
& = \sum_{i=1}^{n}   d_{i} N_{i}(x)
\end{align}
$$
where $c_i$, $d_i$ are weighting coefficients.

With these definitions, we are equipped to rewrite the Galerkin form in an equivalent matrix form.  Substitute for $\tilde{u}$ and $\tilde{w}$ in $\mathbb{G}$:
$$
\begin{align}
\int_{0}^{1} \text{d}x\, \left[
\left( \sum_{i=1}^{n} d_{i} \frac{\text{d}N_{i}}{\text{d}x} \right)
\left( \sum_{j=1}^{n} c_{j} \frac{\text{d}N_{j}}{\text{d}x} + g \frac{\text{d}N_{n+1}}{\text{d}x} \right)
\right]
& =
\int_{0}^{1} \text{d}x\, \left[
\left( \sum_{i=1}^{n} d_{i} N_{i}(x) \right) f
\right]
+ \sum_{i=1}^{n} d_{i} N_{i}(0)
\end{align}
$$

Rewrite this equation as $\sum_{i=1}^{n} d_{i} G_{i}(x) = 0$, where
$$
G_i =
\sum_{j=1}^{n} \left[
\int_{0}^{1} \text{d}x\, c_{j} \frac{\text{d}N_{i}}{\text{d}x} \frac{\text{d}N_{j}}{\text{d}x}
\right]
+ g \left[
\int_{0}^{1} \text{d}x\, \frac{\text{d}N_{i}}{\text{d}x} \frac{\text{d}N_{n+1}}{\text{d}x}
\right]
- \int_{0}^{1} \text{d}x\, N_{i}(x) f
- h N_{i}(0)
\text{.}
$$

Define
$$
\begin{align}
K_{ij} & \equiv \int_{0}^{1} \text{d}x\, \frac{\text{d}N_{i}}{\text{d}x} \frac{\text{d}N_{j}}{\text{d}x}
& i,j \in \left\{1, ..., n\right\} \\
F_{i} & \equiv \int_{0}^{1} \text{d}x\, N_{i}(x) f + h N_{i}(0)
- g \left[
\int_{0}^{1} \text{d}x\, \frac{\text{d}N_{i}}{\text{d}x} \frac{\text{d}N_{n+1}}{\text{d}x}
\right]
& i \in \left\{1, ..., n\right\} \text{.} \\
\end{align}
$$

We can now write the matrix form of the problem, which will be solved using conventional linear-algebraic methods.

<blockquote>
Given the coefficient matrix $\underline{\underline{K}}$ and vector $\underline{F}$, find $\underline{c}$ such that
$$
\underline{\underline{K}} \underline{c} = \underline{F} \text{.}
$$
</blockquote>

### Solution Properties

A solution to either the strong form $\mathbb{S}$ or the weak form $\mathbb{W}$ will satisfy the other; similarly, solutions to either of the Galerkin form $\mathbb{G}$ or the matrix form $\mathbb{M}$ (defined over the same basis functions) will satisfy both.  The primary approximation is the transformation from the weak formulation $\mathbb{W}$ to the approximate Galerkin form $\mathbb{G}$.
$$
\mathbb{S} \leftrightarrow
\mathbb{W} \leftrightsquigarrow
\mathbb{G} \leftrightarrow
\mathbb{M}
$$

The stiffness matrix $\underline{\underline{K}}$ is symmetric, banded, and positive-definite, making it attractive for conventional matrix solution methods.

### Example Solutions

Consider the foregoing system with three nodes and two elements.  We wish to solve the system by hand for a few values of $f$, $g$, $h$.

#### Grid and Shape Functions

<img src="./img/fea/shape-fns-ex1.png" width="200px;" align="right" />
The shape functions are a function of the nodes (here, located equidistant at $x = \left\{0, \frac{1}{2}, 1\right\}$), and not of the equation.
$$
N_1(x) = \left\{
\begin{matrix}
\frac{x-0}{\frac{1}{2}}
\rightarrow &
1-2x
\hspace{1cm} & 0 \leq x \leq \frac{1}{2} \\
0 & & \frac{1}{2} \leq x \leq 1 \\
\end{matrix}
\right.
$$
$$
N_2(x) = \left\{
\begin{matrix}
\frac{\frac{1}{2}-x}{\frac{1}{2}}
\rightarrow &
2x
\hspace{1cm} & 0 \leq x \leq \frac{1}{2} \\
\frac{1-x}{\frac{1}{2}}
\rightarrow &
2-2x
\hspace{1cm} & \frac{1}{2} \leq x \leq 1
\end{matrix}
\right.
$$
$$
N_3(x) = \left\{
\begin{matrix}
0 & & 0 \leq x \leq \frac{1}{2} \\
\frac{x-1}{\frac{1}{2}}
\rightarrow &
2x-2
\hspace{1cm} & \frac{1}{2} \leq x \leq 1 \\
\end{matrix}
\right.
\text{.}
$$

#### Example 1:  $f = 0$, $g = 0$, $h = 1$, $n = 2$
The problem statement reduces to
<blockquote>
Find $u(x)$ such that
$$
\begin{align}
\frac{\text{d}^{2} u(x)}{\text{d} x^{2}} & = 0 & 0<x<1 \\
u(1) & = 0 \\
\left. \frac{\text{d} u}{\text{d} x} \right|_{x=0} & = -1 \\
\end{align}
$$
</blockquote>

To find the resulting matrix formulation $\mathbb{M}$:

The matrix elements of $\underline{\underline{K}}$ are
$$
\begin{align}
K_{11} & = \int_{0}^{1} \text{d}x\, \frac{\text{d}N_{1}}{\text{d}x} \frac{\text{d}N_{1}}{\text{d}x}
= \int_{0}^{\frac{1}{2}} (-2)(-2) = 4x\bigg|_{x=0}^{\frac{1}{2}}
& = 2 \\
K_{12} & = K_{21} = \int_{0}^{1} \text{d}x\, \frac{\text{d}N_{1}}{\text{d}x} \frac{\text{d}N_{2}}{\text{d}x}
= \int_{0}^{\frac{1}{2}} (2)(-2) = -4x\bigg|_{x=0}^{\frac{1}{2}}
& = -2 \\
K_{22} & = \int_{0}^{1} \text{d}x\, \frac{\text{d}N_{2}}{\text{d}x} \frac{\text{d}N_{2}}{\text{d}x}
= \int_{0}^{\frac{1}{2}} (2)(2) + \int_{\frac{1}{2}}^{1} (-2)(-2) = -4x\bigg|_{x=0}^{\frac{1}{2}} + 4x\bigg|_{x=\frac{1}{2}}^{1}
& = 4 \\
\end{align}
$$
thus $\underline{\underline{K}} = \left[ \begin{matrix} 2 & -2 \\ -2 & 4 \end{matrix} \right]$.

The solution vector $\underline{F}$ is
$$
\begin{align}
F_{1} & = \frac{1}{2} N_1(0) = \frac{1}{2} \\
F_{2} & = \frac{1}{2} N_2(0) = 0 \\
\end{align}
$$
and $\underline{F} = \left[ \begin{matrix} 1 \\ 0 \end{matrix} \right]$.

The unknown vector $\underline{c}$ is now found as
$$
\begin{align}
\underline{\underline{K}} \underline{c} & = \underline{F} \Rightarrow \\
\left[ \begin{matrix} 2 & -2 \\ -2 & 4 \end{matrix} \right] \underline{c}
& = \left[ \begin{matrix} 1 \\ 0 \end{matrix} \right] \Rightarrow \\
\underline{c} & = {\left[ \begin{matrix} 2 & -2 \\ -2 & 4 \end{matrix} \right]}^{-1} \times \left[ \begin{matrix} 1 \\ 0 \end{matrix} \right] \\
\underline{c} & = \left[ \begin{matrix} 1 \\ \frac{1}{2} \end{matrix} \right] \text{.}
\end{align}
$$

Substituting the coefficients back into the approximating solution $\tilde{u}$ now yields the FEM solution:
$$
\tilde{u}(x) = \sum_{i=1}^{n} c_{i} N_{i}(x) + g N_{n+1}(x)
= 1 \left\{ \begin{matrix} 1-2x \\ 0 \end{matrix} \right. + \frac{1}{2} \left\{ \begin{matrix} 2x & 0 \leq x \leq \frac{1}{2} \\ 2-2x & \frac{1}{2} \leq x \leq 1 \end{matrix} \right.
= \left\{ \begin{matrix} 1-2x+x \\ 1-x \end{matrix} \right.
= 1-x
$$

The exact solution is $u = 1 - x$, in this case corresponding exactly to the FEM solution.

#### Example 2:  $f \in \mathbb{R}$, $g = 0$, $h = 1$, $n = 2$
For arbitrary real $f$, the problem statement becomes
<blockquote>
Find $u(x)$ such that
$$
\begin{align}
\frac{\text{d}^{2} u(x)}{\text{d} x^{2}} + f & = 0 & 0<x<1 \\
u(1) & = 0 \\
\left. \frac{\text{d} u}{\text{d} x} \right|_{x=0} & = -1 \\
\end{align}
$$
</blockquote>

The nodes have not changed, so the shape functions remain the same.  As before, we find the components of the matrix solution.

The matrix elements of the stiffness matrix $\underline{\underline{K}}$ remain the same, as they are not a function of the source term $f$.

The solution vector $\underline{F}$ becomes
$$
\begin{align}
F_{1} & = \int_{0}^{1} \text{d}x\, N_{1}(x) f + 1 \cdot N_1(0) \\
& = \int_{0}^{\frac{1}{2}} \text{d}x\, (1-2x) f + 1 = 1+\frac{1}{4}f \\
F_{2} & = \int_{0}^{1} \text{d}x\, N_{2}(x) f + 1 \cdot N_2(0) \\
& = \int_{0}^{\frac{1}{2}} \text{d}x\, (2x) f + \int_{\frac{1}{2}}^{1} \text{d}x\, (2-2x) f
= \frac{1}{2} f \\
\end{align}
$$
and $\underline{F} = \left[ \begin{matrix} 1+\frac{1}{4} f \\ \frac{1}{2} f \end{matrix} \right]$.

The unknown vector $\underline{c}$ is now found as
$$
\begin{align}
\underline{\underline{K}} \underline{c} & = \underline{F} \Rightarrow \\
\left[ \begin{matrix} 2 & -2 \\ -2 & 4 \end{matrix} \right] \underline{c}
& = \left[ \begin{matrix} 1+\frac{1}{4} f \\ \frac{1}{2} f \end{matrix} \right] \Rightarrow \\
\underline{c} & = {\left[ \begin{matrix} 2 & -2 \\ -2 & 4 \end{matrix} \right]}^{-1} \times \left[ \begin{matrix} 1+\frac{1}{4} f \\ \frac{1}{2} f \end{matrix} \right] \\
& = {\left[ \begin{matrix} 1 & \frac{1}{2} \\ \frac{1}{2} & \frac{1}{2} \end{matrix} \right]} \times \left[ \begin{matrix} 1+\frac{1}{4} f \\ \frac{1}{2} f \end{matrix} \right] \\
\underline{c} & = \left[ \begin{matrix} 1+\frac{1}{2} f \\ \frac{1}{2}+\frac{3}{8} f \end{matrix} \right] \text{.}
\end{align}
$$

Substituting the coefficients back into the approximating solution $\tilde{u}$ now yields the general FEM solution for the case of $f \in \mathbb{R}$, $g = 0$, $h = 1$, $n = 2$:
$$
\tilde{u}(x) = \sum_{i=1}^{n} c_{i} N_{i}(x) + g N_{n+1}(x)
= \left(1+\frac{1}{2} f\right) \left\{ \begin{matrix} 1-2x \\ 0 \end{matrix} \right. + \left(\frac{1}{2}+\frac{3}{8} f\right) \left\{ \begin{matrix} 2x \\ 2-2x \end{matrix} \right.
= \left\{ \begin{matrix} \left(1+\frac{1}{2}f\right)+\left(-1-\frac{1}{4}f\right)x & 0 \leq x \leq \frac{1}{2} \\ \left(1+\frac{3}{4}f\right)+\left(-1-\frac{3}{4}f\right)x & \frac{1}{2} \leq x \leq 1  \end{matrix} \right. \text{.}
$$

The exact solution is quadratic:
$$
u(x) = -\frac{f}{2}x^{2}-x+\left(1+\frac{f}{2}\right) \text{.}
$$
The FEM solution approximates this solution along the element intervals and matches the exact solution at the nodes.  Some specific cases are plotted in the figure below.  (The black grid represents the approximate solution $\tilde{u}(x)$, and the smooth colored surface represents the real solution $u(x)$.)

<img src="./img/fea/solutions-ex1.png" width="600px;" />

---
### Computer-Enabled FEA

Let us now program and solve the foregoing example explicitly using ANSYS Mechanical APDL and compare the results.  We first need to determine what physics is represented by our purely mathematical consideration above.  Recall the strong form of the problem statement $\mathbb{S}$:
<blockquote>
Given $f$, $g$, $h$, find $u(x)$ such that
$$
\begin{align}
\frac{\text{d}^{2} u(x)}{\text{d} x^{2}} + f & = 0 & 0<x<1 \\
u(1) & = g \\
\left. \frac{\text{d} u}{\text{d} x} \right|_{x=0} & = -h \\
\end{align}
$$
</blockquote>

The second-order differential equation is most like the heat equation $-\alpha\frac{\text{d}^{2} u(x)}{\text{d} x^{2}} + f = 0$ with thermal diffusivity $\alpha \equiv \frac{k}{C_{\text{P}}\rho}=1$ and an arbitrary source term $f$, subject to the same boundary conditions.

-   Start ANSYS Workbench and drag the ![](./img/icon-workbench-apdl.png)`Mechanical APDL` module from the *Toolbox→Component Systems* pane to the *Project Schematic* area.  Right-click on ![](./img/icon-analysis.png)`Analysis` and select `Open Mechanical APDL...`.
    ![](./img/fea/workbench.png)

#### Geometry

-   Click on the the $z$ axis in the lower right of the *Graphics* pane.  The view will align to the $x$–$y$ plane.
    ![](./img/ui-dm-origin.png)
    -   From the *Main Menu*, select `Units→Meter`.
    -   On the *Toolbar* select `Display Plane` ![](./img/icon-dm-display-plane.png) to display a coordinate axis system for the plane.

-   We will now sketch the geometry.  Since ANSYS Mechanical works better with two-dimensional simulations than one-dimensional ones, we will spoof the system into solving our original system as a long thin rectangle rather than as a line.
    
    In the *Tree Outline* pane, select the *Sketching* tab.  The *Sketching Toolboxes* pane replaces the *Tree Outline*.
    -   Under *Constraints*, select `Auto Constraints→Cursor`.  This will cause your cursor to "snap to" a discrete grid, making alignment to the axes more straightforward when constructing a geometry.
    -   Under *Draw*, select the ![](./img/icon-dm-rectangle.png) `Line` tool.  Click on the origin and draw a long flat rectangle to the right approximately $1 \,\text{m}$ in length and $0.01 \,\text{cm}$ in height.  (If you have selected the proper constraints, then hints such as `O` and `C` will appear when your cursor hovers near the origin and axis, respectively.)
        <blockquote>
        Once you have drawn a rectangle, press the ![](./img/icon-best-fit.png)`Best Fit` zoom tool on the *Toolbar* to display the sketch at a more accessible scale.
        </blockquote>
    -   Under *Dimensions*, select the ![](./img/icon-dm-horizontal.png) `Horizontal` tool.  Click to select the $x$ axis at left and then the rightmost point of the line.  Click anywhere above or below the line to establish a dimension line.  In the *Details View* pane, edit the quantity `H1` to `1.0`.
    -   Similarly, use the ![](./img/icon-dm-vertical.png) `Vertical` tool to set the height of the rectangle `V2` to `0.01`.  (If you have to start over, these numbers will be incremented.)
    -   This concludes the sketching phase of creating this geometry.

        ![](./img/fea/sketch.png)

-   Having created the sketch, we are now prepared to instance the part as a model.  In the *Sketching Toolboxes* pane, select the *Modeling* tab.  The *Tree Outline* is displayed in its stead.
    -   Under the ![](./img/ui-dm-plane.png) *XYPlane* entry, there should displayed an entry ![](./img/ui-dm-sketch-complete.png) `Sketch1`.  We will now instance this sketch as a part.
        -   In the *Main Menu* select `Concept→`![](./img/icon-dm-surfaces-from-sketches.png)`Surfaces from Sketches`.  In the *Tree Outline* a new entry ![](./img/ui-dm-surface.png)`SurfaceSk1` now appears.
        -   The *Details View* pane contains a request for data for the entry for `Base Objects`.  Select the rectangle in the *Graphics* pane and then press the `Apply` button.
            <blockquote>
            To multiply select, you can click and hold while dragging from point to point.  Alternatively, you can hold down the `Ctrl` key while clicking multiple times.
            </blockquote>
            The request for information in this entry disappears to be replaced by the text `1`.
        -   Press ![](./img/icon-generate-button.png).  A new entry ![](./img/icon-dm-partbody.png)*1 Part, 1 Body* appears which has a child property ![](./img/ui-dm-surfacebody.png)`Surface Body`.  In addition, the face of the rectangle turns gray.
        -   Now we will extrude the part into the third dimension.  From the *Toolbar*, select ![](./img/icon-dm-extrude.png)`Extrude`.
            -   Select the `Surface Body` as the `Geometry`.
            -   For the `Direction Vector`, select the entire surface (using ![](./img/icon-select-face.png)`Face` selection if necessary).  The `Direction Vector` should read `Face Normal`.
            -   Set the `Depth` to `0.01` m, the same as the height.
        -   Press ![](./img/icon-generate-button.png).
        
        ![](./img/fea/surface.png)

-   ![](./img/icon-save.png)`Save` the project as `fea-intro` and close ![](./img/icon-designmodeler.png)*DesignModeler*.  Note that the status of this step has changed to ![](./img/icon-complete.png)`Complete`.

### Computer-Enabled FEA

Let us now program and solve the foregoing example explicitly using ANSYS Mechanical and compare the results.  We first need to determine what physics is represented by our purely mathematical consideration above.  Recall the strong form of the problem statement $\mathbb{S}$:
<blockquote>
Given $f$, $g$, $h$, find $u(x)$ such that
$$
\begin{align}
\frac{\text{d}^{2} u(x)}{\text{d} x^{2}} + f & = 0 & 0<x<1 \\
u(1) & = g \\
\left. \frac{\text{d} u}{\text{d} x} \right|_{x=0} & = -h \\
\end{align}
$$
</blockquote>

The second-order differential equation is most like the heat equation $-\alpha\frac{\text{d}^{2} u(x)}{\text{d} x^{2}} + f = 0$ with thermal diffusivity $\alpha \equiv \frac{k}{C_{\text{P}}\rho}=1$ and an arbitrary source term $f$, subject to the same boundary conditions.

-   Start ANSYS Workbench and drag the ![](./img/icon-workbench-ss-thermal.png)`Steady-State Thermal` module from the *Toolbox* pane to the *Project Schematic* area.  Double-click on ![](./img/icon-designmodeler.png)`Geometry` and enter ![](./img/icon-designmodeler.png)*DesignModeler*.
    ![](./img/fea/workbench.png)

#### Geometry

-   Click on the the $z$ axis in the lower right of the *Graphics* pane.  The view will align to the $x$–$y$ plane.
    ![](./img/ui-dm-origin.png)
    -   From the *Main Menu*, select `Units→Meter`.
    -   On the *Toolbar* select `Display Plane` ![](./img/icon-dm-display-plane.png) to display a coordinate axis system for the plane.

-   We will now sketch the geometry.  Since ANSYS Mechanical works better with two-dimensional simulations than one-dimensional ones, we will spoof the system into solving our original system as a long thin rectangle rather than as a line.
    
    In the *Tree Outline* pane, select the *Sketching* tab.  The *Sketching Toolboxes* pane replaces the *Tree Outline*.
    -   Under *Constraints*, select `Auto Constraints→Cursor`.  This will cause your cursor to "snap to" a discrete grid, making alignment to the axes more straightforward when constructing a geometry.
    -   Under *Draw*, select the ![](./img/icon-dm-rectangle.png) `Line` tool.  Click on the origin and draw a long flat rectangle to the right approximately $1 \,\text{m}$ in length and $0.01 \,\text{cm}$ in height.  (If you have selected the proper constraints, then hints such as `O` and `C` will appear when your cursor hovers near the origin and axis, respectively.)
        <blockquote>
        Once you have drawn a rectangle, press the ![](./img/icon-best-fit.png)`Best Fit` zoom tool on the *Toolbar* to display the sketch at a more accessible scale.
        </blockquote>
    -   Under *Dimensions*, select the ![](./img/icon-dm-horizontal.png) `Horizontal` tool.  Click to select the $x$ axis at left and then the rightmost point of the line.  Click anywhere above or below the line to establish a dimension line.  In the *Details View* pane, edit the quantity `H1` to `1.0`.
    -   Similarly, use the ![](./img/icon-dm-vertical.png) `Vertical` tool to set the height of the rectangle `V2` to `0.01`.  (If you have to start over, these numbers will be incremented.)
    -   This concludes the sketching phase of creating this geometry.

        ![](./img/fea/sketch.png)

-   Having created the sketch, we are now prepared to instance the part as a model.  In the *Sketching Toolboxes* pane, select the *Modeling* tab.  The *Tree Outline* is displayed in its stead.
    -   Under the ![](./img/ui-dm-plane.png) *XYPlane* entry, there should displayed an entry ![](./img/ui-dm-sketch-complete.png) `Sketch1`.  We will now instance this sketch as a part.
        -   In the *Main Menu* select `Concept→`![](./img/icon-dm-surfaces-from-sketches.png)`Surfaces from Sketches`.  In the *Tree Outline* a new entry ![](./img/ui-dm-surface.png)`SurfaceSk1` now appears.
        -   The *Details View* pane contains a request for data for the entry for `Base Objects`.  Select the rectangle in the *Graphics* pane and then press the `Apply` button.
            <blockquote>
            To multiply select, you can click and hold while dragging from point to point.  Alternatively, you can hold down the `Ctrl` key while clicking multiple times.
            </blockquote>
            The request for information in this entry disappears to be replaced by the text `1`.
        -   Press ![](./img/icon-generate-button.png).  A new entry ![](./img/icon-dm-partbody.png)*1 Part, 1 Body* appears which has a child property ![](./img/ui-dm-surfacebody.png)`Surface Body`.  In addition, the face of the rectangle turns gray.
        -   Now we will extrude the part into the third dimension.  From the *Toolbar*, select ![](./img/icon-dm-extrude.png)`Extrude`.
            -   Select the `Surface Body` as the `Geometry`.
            -   For the `Direction Vector`, select the entire surface (using ![](./img/icon-select-face.png)`Face` selection if necessary).  The `Direction Vector` should read `Face Normal`.
            -   Set the `Depth` to `0.01` m, the same as the height.
        -   Press ![](./img/icon-generate-button.png).
        
        ![](./img/fea/surface.png)

-   ![](./img/icon-save.png)`Save` the project as `fea-intro` and close ![](./img/icon-designmodeler.png)*DesignModeler*.  Note that the status of this step has changed to ![](./img/icon-complete.png)`Complete`.

#### Model

Next we will define the nodes over which the problem will be solved.

-   Select ![](./img/icon-model.png)`Model`.  The ![](./img/icon-mesh.png)*Mechanical Meshing* tool opens.

-   As before, align to the $x$–$y$ plane in the *Geometry* pane and ensure that the length scale of the system is the meter.

-   The tiny lightning bolt ![](./img/icon-generate-small.png) indicates that the ![](./img/ui-mesh.png)`Mesh` needs to be created.  In the *Toolbar*, press ![](./img/icon-update-button.png) to see what the default mesh settings generate.
    
    This mesh would probably be inadequate for a real simulation; in our case, since we're comparing with a very simple solution by hand, it's actually _too_ resolved.

-   In order to have a properly behaved mesh, we need to specify two things:  first, that the mesh will align in a certain way with the edges (![](./img/icon-mapped-face-meshing.png)`Mapped Face Meshing`) and that the mesh will have a certain number of divisions (![](./img/icon-sizing.png)`Sizing`).

    -   Right-click on `Mesh` and select `Insert→`![](./img/icon-mapped-face-meshing.png)`Mapped Face Meshing`.  Under ![](./img/ui-mesh.png)`Mesh` in the *Outline* pane, an entry ![](./img/icon-mapped-face-meshing.png)`Mapped Face Meshing` appears.
        -   In the *Details* pane, a `Geometry` is requested as a basis for the sizing.  Using the *Toolbar*'s ![](./img/icon-select-body.png)`Body` selection tool, select the rectangle as the basis and press `Apply`.
    
    -   Right-click on `Mesh` and select `Insert→`![](./img/icon-sizing.png)`Sizing`.  Under ![](./img/ui-mesh.png)`Mesh` in the *Outline* pane, an entry ![](./img/icon-sizing.png)`Sizing` appears.
        -   In the *Details* Pane, a `Geometry` is requested as a basis for the sizing.  Using the *Toolbar*'s ![](./img/icon-select-edge.png)`Edge` selection tool, select the top line as the basis and press `Apply`.  The type of sizing is updated to `Edge Sizing` and a bold blue marker appears in the *Geometry* viewport.
            ![](./img/fea/mesh-sizing.png)
        -   In the *Details* pane, change the `Type` to `Number of Divisions` and specify `2` divisions.  This should create two nodes when you ![](./img/icon-update-button.png) the mesh.

-   We will next create named selections, which are collections of nodes that we can utilize to define loads and boundary conditions.
    -   Use the ![](./img/icon-select-vertex.png)`Vertex/Node` selection tool to select the leftmost nodes.
    -   Right-click on the nodes and select ![](./img/icon-create-named-selection.png)`Create Named Selection`.  Name the selection `lhs`.  A new entry under the ![](./img/ui-model.png)*Model* tree at left appears, entitled ![](./img/ui-named-selections.png)*Named Selections*.
    -   Similarly, create a named selection for the rightmost nodes entitled `rhs`.

-   Select `File→`![](./img/icon-save.png)`Save Project...`.

#### Setup

With ANSYS, we do not need to close the meshing tool as we can define the boundary conditions in the same environment.

-   Under ![](./img/ui-ss-thermal.png)*Steady-State Thermal*, select ![](./img/icon-initial-temperature.png)`Initial Temperature`.
    -   In the *Details* pane, set the `Initial Temperature Value` to `0`.

##### $g = 0$
We will now define the boundary conditions applying to the nodes at the left-hand and right-hand sides.

-   Right-click on ![](./img/ui-analysis-settings.png)`Analysis Settings` and `Insert→`![](./img/icon-temperature.png)`Temperature`.  Specify the nodes at the right-hand edge as the `Geometry` and set the `Type` to `Temperature` and the `Magnitude` to `0`°C.

##### $h = 1$
The left-hand boundary condition is effectively $k\frac{\text{d}u}{\text{d}x} = -h$ and thus the units work out properly if we redefine $k$ (which we do in the next section).  (Strictly speaking, we will specify a total heat flow out of the edge due to the way ANSYS Mechanical handles boundary conditions in two dimensions.)

-   Right-click on ![](./img/ui-analysis-settings.png)`Analysis Settings` and `Insert→`![](./img/icon-heat-flow.png)`Temperature`.  Specify the nodes at the left-hand edge as the `Geometry` and set the `Type` to `Heat Flow` and the `Magnitude` to `-1` W.

##### $f = 2$
-   The $f$ term physically corresponds to heat generation within the object.  Thus `Insert→`![](./img/icon-internal-heat-generation.png)`Internal Heat Generation` applied to the entire line with a magnitude of $2 \,\text{W}\cdot\text{m}^{-3}$.

-   Press the ![](./img/icon-show-mesh.png)`Show Mesh` button to display the boundary conditions in context.

    ![](./img/fea/bcs.png)

-   ![](./img/icon-save.png)`Save` the project and close the *Setup* window.

#### Engineering Data

-   Double-click to open the ![](./img/icon-engineering-data.png)`Engineering Data` module, which opens as a new tab within the *Workbench* interface.

-   Rather than create a new material, for the purposes of this simulation we will simply redefine `Structural Steel` to have a `Thermal Conductivity` of `1` and `0`°C.

    ![](./img/fea/engg-data.png)
    
    <blockquote>
    Fortunately, the materials data library is linked into ANSYS, so that your changes here do not impact the original data.  To recover the original thermal conductivity, for instance, right-click `Structural Steel` and select ![](./img/icon-refresh-from-linked-source.png)`Refresh from linked source`.
    </blockquote>

-   This effectively rescales our equations and boundary conditions to the original problem specification.  ![](./img/icon-save.png)`Save` the project and close the *Engineering Data* tab.

### Computer-Enabled FEA

Let us now program and solve the foregoing example explicitly using ANSYS Mechanical and compare the results.  We first need to determine what physics is represented by our purely mathematical consideration above.  Recall the strong form of the problem statement $\mathbb{S}$:
<blockquote>
Given $f$, $g$, $h$, find $u(x)$ such that
$$
\begin{align}
\frac{\text{d}^{2} u(x)}{\text{d} x^{2}} + f & = 0 & 0<x<1 \\
u(1) & = g \\
\left. \frac{\text{d} u}{\text{d} x} \right|_{x=0} & = -h \\
\end{align}
$$
</blockquote>

The second-order differential equation is most like the heat equation $-\alpha\frac{\text{d}^{2} u(x)}{\text{d} x^{2}} + f = 0$ with thermal diffusivity $\alpha \equiv \frac{k}{C_{\text{P}}\rho}=1$ and an arbitrary source term $f$, subject to the same boundary conditions.

-   Start ANSYS Workbench and drag the ![](./img/icon-workbench-ss-thermal.png)`Steady-State Thermal` module from the *Toolbox* pane to the *Project Schematic* area.  Double-click on ![](./img/icon-designmodeler.png)`Geometry` and enter ![](./img/icon-designmodeler.png)*DesignModeler*.
    ![](./img/fea/workbench.png)

#### Geometry

-   Click on the the $z$ axis in the lower right of the *Graphics* pane.  The view will align to the $x$–$y$ plane.
    ![](./img/ui-dm-origin.png)
    -   From the *Main Menu*, select `Units→Meter`.
    -   On the *Toolbar* select `Display Plane` ![](./img/icon-dm-display-plane.png) to display a coordinate axis system for the plane.

-   We will now sketch the geometry.  In the *Tree Outline* pane, select the *Sketching* tab.  The *Sketching Toolboxes* pane replaces the *Tree Outline*.
    -   Under *Constraints*, select `Auto Constraints→Cursor`.  This will cause your cursor to "snap to" a discrete grid, making alignment to the axes more straightforward when constructing a geometry.
    -   Under *Draw*, select the ![](./img/icon-dm-line.png) `Line` tool.  Click on the origin and draw a horizontal line to the right approximately $1 \,\text{m}$ in length.  (If you have selected the proper constraints, then hints such as `O` and `C` will appear when your cursor hovers near the origin and axis, respectively.)
    -   Under *Dimensions*, select the ![](./img/icon-dm-horizontal.png) `Horizontal` tool.  Click to select the $x$ axis at left and then the rightmost point of the line.  Click anywhere above or below the line to establish a dimension line.  In the *Details View* pane, edit the quantity `H1` to $1 \,\text{m}$.
    -   This concludes the sketching phase of creating this geometry.

-   Having created the sketch, we are now prepared to instance the part as a model.  In the *Sketching Toolboxes* pane, select the *Modeling* tab.  The *Tree Outline* is displayed in its stead.
    -   Under the ![](./img/ui-dm-plane.png) *XYPlane* entry, there should displayed an entry ![](./img/ui-dm-sketch-complete.png) `Sketch1`.  We will now instance this sketch as a part.
        -   In the *Main Menu* select `Concept→`![](./img/icon-dm-lines-from-points.png)`Lines from Points`.  In the *Tree Outline* a new entry ![](./img/ui-dm-line.png)`Line1` now appears.
        -   The *Details View* pane contains a request for data for the entry for `Point Segments`.  Select the left and right endpoints of the line segment in the *Graphics* pane and then press the `Apply` button.
            <blockquote>
            To multiply select, you can click and hold while dragging from point to point.  Alternatively, you can hold down the `Ctrl` key while clicking multiple times.
            </blockquote>
            The request for information in this entry disappears to be replaced by the text `1`.
        -   Press ![](./img/icon-generate-button.png).  A new entry ![](./img/icon-dm-partbody.png)*1 Part, 1 Body* appears which has a child property ![](./img/icon-dm-linebody.png)`Line Body`.

-   ![](./img/icon-save.png)`Save` the project as `fea-intro` and close ![](./img/icon-designmodeler.png)*DesignModeler*.  Note that the status of this step has changed to ![](./img/icon-complete.png)`Complete`.

#### Model

Next we will define the nodes over which the problem will be solved.

-   Select ![](./img/icon-model.png)`Model`.  The ![](./img/icon-mesh.png)*Mechanical Meshing* tool opens.

-   As before, align to the $x$–$y$ plane in the *Geometry* pane and ensure that the length scale of the system is the meter.

-   The tiny lightning bolt ![](./img/icon-generate-small.png) indicates that the ![](./img/ui-mesh.png)`Mesh` needs to be created.  In the *Toolbar*, press ![](./img/icon-update-button.png) to see what the default mesh settings generate.
    ![](./img/fea/mesh-default.png)
    This mesh would probably be inadequate for a real simulation; in our case, since we're comparing with a very simple solution by hand, it's actually _too_ resolved.

-   Right-click on `Mesh` and select `Insert→`![](./img/icon-sizing.png)`Sizing`.  Under ![](./img/ui-mesh.png)`Mesh` in the *Outline* pane, an entry ![](./img/icon-sizing.png)`Sizing` appears.
    -   In the *Details* Pane, a `Geometry` is requested as a basis for the sizing.  Using the *Toolbar*'s ![](./img/icon-select-edge.png)`Edge` selection tool, select the line as the basis and press `Apply`.  The type of sizing is updated to `Edge Sizing` and a bold blue marker appears in the *Geometry* viewport.
        ![](./img/fea/mesh-sizing.png)
    -   In the *Details* pane, change the `Type` to `Number of Divisions` and specify `2` divisions.  This should create two nodes when you ![](./img/icon-update-button.png) the mesh.

-   We will next create named selections, which are collections of nodes that we can utilize to define loads and boundary conditions.
    -   Use the ![](./img/icon-select-vertex.png)`Vertex/Node` selection tool to select the leftmost node.
    -   Right-click on the node and select ![](./img/icon-create-named-selection.png)`Create Named Selection`.  Name the selection `lhs`.  A new entry under the ![](./img/ui-model.png)*Model* tree at left appears, entitled ![](./img/ui-named-selections.png)*Named Selections*.
    -   Similarly, create a named selection for the rightmost point entitled `rhs`.

-   Select `File→`![](./img/icon-save.png)`Save Project...`.

#### Setup

With ANSYS, we do not need to close the meshing tool as we can define the boundary conditions in the same environment.

-   Under ![](./img/ui-ss-thermal.png)*Steady-State Thermal*, select ![](./img/icon-initial-temperature.png)`Initial Temperature`.
    -   In the *Details* pane, set the `Initial Temperature Value` to `0`.

##### $g = 0$
-   We will now define the boundary conditions applying to the nodes at the left-hand and right-hand sides.
    -   Right-click on ![](./img/ui-analysis-settings.png)`Analysis Settings` and `Insert→`![](./img/icon-temperature.png)`Temperature`.  Specify the node at $(1,0)$ as the `Geometry` and set the `Type` to `Temperature` and the `Magnitude` to `0`°C.

##### $h = 1$
Due to a quirk of how ANSYS Mechanical handles boundary conditions, a heat flux boundary condition isn't supported in one dimension.  We will thus cheat a little and use the known left-hand temperature value of $2$ from our prior hand calculation.  (Bear in mind that this boundary condition is effectively $k\frac{\text{d}u}{\text{d}x} = -h$ and thus the units work out properly.)

-   Apply the boundary condition to the node at $(0,0)$.

##### $f = 2$
-   The $f$ term physically corresponds to heat generation within the object.  Thus `Insert→`![](./img/icon-internal-heat-generation.png)`Internal Heat Generation` applied to the entire line with a magnitude of $2 \,\text{W}\cdot\text{m}^{-3}$.

-   Press the ![](./img/icon-show-mesh.png)`Show Mesh` button to display the boundary conditions in context.

    ![](./img/fea/bcs.png)

-   ![](./img/icon-save.png)`Save` the project and close the *Setup* window.

### Computer-Enabled FEA

Let us now program and solve the foregoing example explicitly using ANSYS Mechanical and compare the results.  We first need to determine what physics is represented by our purely mathematical consideration above.  Recall the strong form of the problem statement $\mathbb{S}$:
<blockquote>
Given $f$, $g$, $h$, find $u(x)$ such that
$$
\begin{align}
\frac{\text{d}^{2} u(x)}{\text{d} x^{2}} + f & = 0 & 0<x<1 \\
u(1) & = g \\
\left. \frac{\text{d} u}{\text{d} x} \right|_{x=0} & = -h \\
\end{align}
$$
</blockquote>

The second-order differential equation is most like the heat equation $-\alpha\frac{\text{d}^{2} u(x)}{\text{d} x^{2}} + f = 0$ with thermal diffusivity $\alpha \equiv \frac{k}{C_{\text{P}}\rho}=1$ and an arbitrary source term $f$, subject to the same boundary conditions.

-   Start ANSYS Workbench and drag the ![](./img/icon-workbench-ss-thermal.png)`Steady-State Thermal` module from the *Toolbox* pane to the *Project Schematic* area.  Double-click on ![](./img/icon-designmodeler.png)`Geometry` and enter ![](./img/icon-designmodeler.png)*DesignModeler*.
    ![](./img/fea/workbench.png)

#### Geometry

-   Click on the the $z$ axis in the lower right of the *Graphics* pane.  The view will align to the $x$–$y$ plane.
    ![](./img/ui-dm-origin.png)
    -   From the *Main Menu*, select `Units→Meter`.
    -   On the *Toolbar* select `Display Plane` ![](./img/icon-dm-display-plane.png) to display a coordinate axis system for the plane.

-   We will now sketch the geometry.  Since ANSYS Mechanical works better with two-dimensional simulations than one-dimensional ones, we will spoof the system into solving our original system as a long thin rectangle rather than as a line.
    
    In the *Tree Outline* pane, select the *Sketching* tab.  The *Sketching Toolboxes* pane replaces the *Tree Outline*.
    -   Under *Constraints*, select `Auto Constraints→Cursor`.  This will cause your cursor to "snap to" a discrete grid, making alignment to the axes more straightforward when constructing a geometry.
    -   Under *Draw*, select the ![](./img/icon-dm-rectangle.png) `Line` tool.  Click on the origin and draw a long flat rectangle to the right approximately $1 \,\text{m}$ in length and $0.01 \,\text{cm}$ in height.  (If you have selected the proper constraints, then hints such as `O` and `C` will appear when your cursor hovers near the origin and axis, respectively.)
    -   Under *Dimensions*, select the ![](./img/icon-dm-horizontal.png) `Horizontal` tool.  Click to select the $x$ axis at left and then the rightmost point of the line.  Click anywhere above or below the line to establish a dimension line.  In the *Details View* pane, edit the quantity `H1` to `1.0`.
    -   Similarly, use the ![](./img/icon-dm-vertical.png) `Vertical` tool to set the height of the rectangle `V2` to `0.01`.  (If you have to start over, these numbers will be incremented.)
    -   This concludes the sketching phase of creating this geometry.

        ![](./img/fea/sketch.png)

-   Having created the sketch, we are now prepared to instance the part as a model.  In the *Sketching Toolboxes* pane, select the *Modeling* tab.  The *Tree Outline* is displayed in its stead.
    -   Under the ![](./img/ui-dm-plane.png) *XYPlane* entry, there should displayed an entry ![](./img/ui-dm-sketch-complete.png) `Sketch1`.  We will now instance this sketch as a part.
        -   In the *Main Menu* select `Concept→`![](./img/icon-dm-surfaces-from-sketches.png)`Surfaces from Sketches`.  In the *Tree Outline* a new entry ![](./img/ui-dm-surface.png)`SurfaceSk1` now appears.
        -   The *Details View* pane contains a request for data for the entry for `Base Objects`.  Select the rectangle in the *Graphics* pane and then press the `Apply` button.
            <blockquote>
            To multiply select, you can click and hold while dragging from point to point.  Alternatively, you can hold down the `Ctrl` key while clicking multiple times.
            </blockquote>
            The request for information in this entry disappears to be replaced by the text `1`.
        -   Press ![](./img/icon-generate-button.png).  A new entry ![](./img/icon-dm-partbody.png)*1 Part, 1 Body* appears which has a child property ![](./img/ui-dm-surfacebody.png)`Surface Body`.  In addition, the face of the rectangle turns gray.
        
        ![](./img/fea/surface.png)

-   ![](./img/icon-save.png)`Save` the project as `fea-intro` and close ![](./img/icon-designmodeler.png)*DesignModeler*.  Note that the status of this step has changed to ![](./img/icon-complete.png)`Complete`.

#### Model

Next we will define the nodes over which the problem will be solved.

-   Select ![](./img/icon-model.png)`Model`.  The ![](./img/icon-mesh.png)*Mechanical Meshing* tool opens.

-   As before, align to the $x$–$y$ plane in the *Geometry* pane and ensure that the length scale of the system is the meter.

-   The tiny lightning bolt ![](./img/icon-generate-small.png) indicates that the ![](./img/ui-mesh.png)`Mesh` needs to be created.  In the *Toolbar*, press ![](./img/icon-update-button.png) to see what the default mesh settings generate.
    
    This mesh would probably be inadequate for a real simulation; in our case, since we're comparing with a very simple solution by hand, it's actually _too_ resolved.

-   In order to have a properly behaved mesh, we need to specify two things:  first, that the mesh will align in a certain way with the edges (![](./img/icon-mapped-face-meshing.png)`Mapped Face Meshing`) and that the mesh will have a certain number of divisions (![](./img/icon-sizing.png)`Sizing`).

    -   Right-click on `Mesh` and select `Insert→`![](./img/icon-mapped-face-meshing.png)`Mapped Face Meshing`.  Under ![](./img/ui-mesh.png)`Mesh` in the *Outline* pane, an entry ![](./img/icon-mapped-face-meshing.png)`Mapped Face Meshing` appears.
        -   In the *Details* pane, a `Geometry` is requested as a basis for the sizing.  Using the *Toolbar*'s ![](./img/icon-select-face.png)`Face` selection tool, select the rectangle as the basis and press `Apply`.
    
    -   Right-click on `Mesh` and select `Insert→`![](./img/icon-sizing.png)`Sizing`.  Under ![](./img/ui-mesh.png)`Mesh` in the *Outline* pane, an entry ![](./img/icon-sizing.png)`Sizing` appears.
        -   In the *Details* Pane, a `Geometry` is requested as a basis for the sizing.  Using the *Toolbar*'s ![](./img/icon-select-edge.png)`Edge` selection tool, select the top line as the basis and press `Apply`.  The type of sizing is updated to `Edge Sizing` and a bold blue marker appears in the *Geometry* viewport.
            ![](./img/fea/mesh-sizing.png)
        -   In the *Details* pane, change the `Type` to `Number of Divisions` and specify `2` divisions.  This should create two nodes when you ![](./img/icon-update-button.png) the mesh.

-   We will next create named selections, which are collections of nodes that we can utilize to define loads and boundary conditions.
    -   Use the ![](./img/icon-select-vertex.png)`Vertex/Node` selection tool to select the leftmost nodes.
    -   Right-click on the nodes and select ![](./img/icon-create-named-selection.png)`Create Named Selection`.  Name the selection `lhs`.  A new entry under the ![](./img/ui-model.png)*Model* tree at left appears, entitled ![](./img/ui-named-selections.png)*Named Selections*.
    -   Similarly, create a named selection for the rightmost nodes entitled `rhs`.

-   Select `File→`![](./img/icon-save.png)`Save Project...`.